# Lab 4: Spark SQL

## 🎯 **Learning Objectives:**
- Master Spark SQL for querying data
- Learn to use SQL syntax with Spark
- Understand CREATE TABLE, VIEW, and UDF
- Practice advanced SQL operations
- Learn catalog management

## 📚 **Key Concepts:**
1. **Spark SQL**: SQL interface for Spark
2. **CREATE TABLE**: Managed and external tables
3. **VIEW**: Virtual tables from queries
4. **UDF**: User Defined Functions
5. **Catalog**: Metadata management

## 🏗️ **Architecture Overview:**
```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   DataFrames    │───▶│   Spark SQL      │───▶│   SQL Results   │
│   (Python API)  │    │   (SQL Interface)│    │                 │
└─────────────────┘    └──────────────────┘    └─────────────────┘
         │                        │                        │
         ▼                        ▼                        ▼
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│ Register as     │    │ SQL Queries      │    │ Tables & Views  │
│ Temporary View  │    │ • SELECT         │    │ • Managed       │
│                 │    │ • JOIN          │    │ • External      │
│                 │    │ • Window Funcs  │    │ • Views         │
└─────────────────┘    └──────────────────┘    └─────────────────┘
```

## 📊 **Use Cases:**
- **SQL Queries**: Query DataFrames using SQL syntax
- **Table Management**: Create and manage tables
- **View Creation**: Create reusable views
- **Custom Functions**: Extend SQL with UDFs
- **Catalog Operations**: Manage database catalogs


In [ ]:
# Install and Import Dependencies
%pip install pyspark findspark pandas numpy pyarrow psycopg2-binary sqlalchemy

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

print("✅ Dependencies installed and imported successfully!")


In [ ]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("SparkSQLLab") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

# Set log level to reduce verbosity
spark.sparkContext.setLogLevel("WARN")

print("🚀 Spark SQL Session initialized successfully!")
print(f"📊 Spark Version: {spark.version}")
print(f"🔗 Master URL: {spark.sparkContext.master}")
print(f"💾 Default Catalog: {spark.catalog.currentCatalog()}")
print(f"📁 Current Database: {spark.catalog.currentDatabase()}")


## Exercise 1: Basic Spark SQL Queries

### 🎯 **Learning Objectives:**
- Query DataFrames using SQL syntax
- Register DataFrames as temporary views
- Perform basic SQL operations
- Understand SQL vs DataFrame API


In [ ]:
# Create Sample Data
print("📊 Creating sample datasets for Spark SQL Lab...")

# Sample Sales Data
sales_data = []
products = ['Laptop', 'Phone', 'Tablet', 'Headphones', 'Camera', 'Monitor', 'Keyboard', 'Mouse']
customers = ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve', 'Frank', 'Grace', 'Henry']
categories = ['Electronics', 'Accessories', 'Computing']

for i in range(1000):
    sales_data.append({
        'sale_id': f'SALE_{i+1:04d}',
        'customer_name': random.choice(customers),
        'product_name': random.choice(products),
        'category': random.choice(categories),
        'quantity': random.randint(1, 5),
        'unit_price': round(random.uniform(50, 2000), 2),
        'sale_date': (datetime.now() - timedelta(days=random.randint(0, 365))).strftime('%Y-%m-%d'),
        'region': random.choice(['North', 'South', 'East', 'West', 'Central'])
    })

# Sample Customer Data
customer_data = []
for customer in customers:
    customer_data.append({
        'customer_name': customer,
        'age': random.randint(25, 65),
        'city': random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']),
        'membership_level': random.choice(['Bronze', 'Silver', 'Gold', 'Platinum']),
        'join_date': (datetime.now() - timedelta(days=random.randint(30, 1000))).strftime('%Y-%m-%d'),
        'total_purchases': random.randint(5, 50)
    })

# Create DataFrames
sales_df = spark.createDataFrame(sales_data)
customers_df = spark.createDataFrame(customer_data)

print(f"✅ Sample data created:")
print(f"   📊 Sales records: {len(sales_data)}")
print(f"   👥 Customer records: {len(customer_data)}")


In [ ]:
# Register DataFrames as Temporary Views
print("📝 Registering DataFrames as temporary views for SQL queries...")

sales_df.createOrReplaceTempView("sales")
customers_df.createOrReplaceTempView("customers")

print("✅ Views created:")
print("   - sales")
print("   - customers")

# List all temporary views
print("\n📋 Available temporary views:")
spark.sql("SHOW TABLES").show()


In [ ]:
# Basic SQL Queries
print("🔍 Exercise 1: Basic Spark SQL Queries")
print("=" * 60)

print("\n1️⃣ Simple SELECT query:")
result1 = spark.sql("""
    SELECT 
        sale_id,
        customer_name,
        product_name,
        quantity,
        unit_price,
        (quantity * unit_price) as total_amount
    FROM sales
    LIMIT 10
""")
result1.show(truncate=False)

print("\n2️⃣ Filtering with WHERE clause:")
result2 = spark.sql("""
    SELECT *
    FROM sales
    WHERE category = 'Electronics'
        AND quantity > 2
    ORDER BY unit_price DESC
    LIMIT 10
""")
result2.show(truncate=False)

print("\n3️⃣ Aggregations with GROUP BY:")
result3 = spark.sql("""
    SELECT 
        product_name,
        COUNT(*) as sale_count,
        SUM(quantity) as total_quantity,
        AVG(unit_price) as avg_price,
        SUM(quantity * unit_price) as total_revenue
    FROM sales
    GROUP BY product_name
    ORDER BY total_revenue DESC
""")
result3.show(truncate=False)


## Exercise 2: JOIN Operations with SQL

### 🎯 **Learning Objectives:**
- Perform JOINs using SQL syntax
- Understand different JOIN types
- Combine multiple tables


In [ ]:
# JOIN Operations with SQL
print("🔗 Exercise 2: JOIN Operations with SQL")
print("=" * 60)

print("\n1️⃣ INNER JOIN:")
result4 = spark.sql("""
    SELECT 
        s.sale_id,
        s.customer_name,
        s.product_name,
        s.quantity,
        s.unit_price,
        c.city,
        c.membership_level,
        c.age
    FROM sales s
    INNER JOIN customers c
        ON s.customer_name = c.customer_name
    LIMIT 10
""")
result4.show(truncate=False)

print("\n2️⃣ LEFT JOIN:")
result5 = spark.sql("""
    SELECT 
        c.customer_name,
        c.city,
        c.membership_level,
        COUNT(s.sale_id) as total_sales,
        COALESCE(SUM(s.quantity * s.unit_price), 0) as total_spent
    FROM customers c
    LEFT JOIN sales s
        ON c.customer_name = s.customer_name
    GROUP BY c.customer_name, c.city, c.membership_level
    ORDER BY total_spent DESC
""")
result5.show(truncate=False)

print("\n3️⃣ Complex JOIN with Aggregations:")
result6 = spark.sql("""
    SELECT 
        s.region,
        s.category,
        COUNT(DISTINCT s.customer_name) as unique_customers,
        COUNT(*) as total_sales,
        SUM(s.quantity * s.unit_price) as total_revenue,
        AVG(s.quantity * s.unit_price) as avg_transaction_value
    FROM sales s
    GROUP BY s.region, s.category
    ORDER BY total_revenue DESC
""")
result6.show(truncate=False)


In [ ]:
# Window Functions with SQL
print("🪟 Exercise 3: Window Functions with SQL")
print("=" * 60)

print("\n1️⃣ ROW_NUMBER and RANK:")
result7 = spark.sql("""
    SELECT 
        customer_name,
        product_name,
        quantity * unit_price as total_amount,
        ROW_NUMBER() OVER (
            PARTITION BY customer_name 
            ORDER BY quantity * unit_price DESC
        ) as row_num,
        RANK() OVER (
            PARTITION BY customer_name 
            ORDER BY quantity * unit_price DESC
        ) as rank,
        DENSE_RANK() OVER (
            PARTITION BY customer_name 
            ORDER BY quantity * unit_price DESC
        ) as dense_rank
    FROM sales
    WHERE customer_name = 'Alice'
    ORDER BY total_amount DESC
""")
result7.show(truncate=False)

print("\n2️⃣ Running Totals:")
result8 = spark.sql("""
    SELECT 
        sale_id,
        customer_name,
        sale_date,
        quantity * unit_price as transaction_amount,
        SUM(quantity * unit_price) OVER (
            PARTITION BY customer_name 
            ORDER BY sale_date 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) as running_total
    FROM sales
    WHERE customer_name = 'Alice'
    ORDER BY sale_date
    LIMIT 10
""")
result8.show(truncate=False)

print("\n3️⃣ Moving Average:")
result9 = spark.sql("""
    SELECT 
        sale_date,
        product_name,
        quantity * unit_price as daily_revenue,
        AVG(quantity * unit_price) OVER (
            PARTITION BY product_name 
            ORDER BY sale_date 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) as moving_avg_3days
    FROM sales
    WHERE product_name = 'Laptop'
    ORDER BY sale_date
    LIMIT 10
""")
result9.show(truncate=False)


In [ ]:
# CREATE VIEW
print("👁️ Exercise 4: CREATE VIEW")
print("=" * 60)

print("\n1️⃣ Create a temporary view for customer sales summary:")
spark.sql("""
    CREATE OR REPLACE TEMP VIEW customer_sales_summary AS
    SELECT 
        c.customer_name,
        c.city,
        c.membership_level,
        COUNT(s.sale_id) as total_orders,
        SUM(s.quantity * s.unit_price) as total_spent,
        AVG(s.quantity * s.unit_price) as avg_order_value,
        MAX(s.sale_date) as last_purchase_date
    FROM customers c
    LEFT JOIN sales s
        ON c.customer_name = s.customer_name
    GROUP BY c.customer_name, c.city, c.membership_level
""")

print("✅ View 'customer_sales_summary' created!")

# Query the view
print("\n2️⃣ Query the view:")
result10 = spark.sql("""
    SELECT *
    FROM customer_sales_summary
    ORDER BY total_spent DESC
""")
result10.show(truncate=False)

print("\n3️⃣ Create another view for product performance:")
spark.sql("""
    CREATE OR REPLACE TEMP VIEW product_performance AS
    SELECT 
        product_name,
        category,
        COUNT(*) as sale_count,
        SUM(quantity) as total_quantity_sold,
        SUM(quantity * unit_price) as total_revenue,
        AVG(unit_price) as avg_price,
        MIN(unit_price) as min_price,
        MAX(unit_price) as max_price
    FROM sales
    GROUP BY product_name, category
""")

print("✅ View 'product_performance' created!")

# Query the new view
result11 = spark.sql("""
    SELECT *
    FROM product_performance
    ORDER BY total_revenue DESC
""")
result11.show(truncate=False)

# List all views
print("\n📋 All available views:")
spark.sql("SHOW TABLES").show()


In [ ]:
# CREATE TABLE
print("📊 Exercise 5: CREATE TABLE")
print("=" * 60)

print("\n1️⃣ Create a managed table from query result:")
spark.sql("""
    CREATE OR REPLACE TABLE sales_summary AS
    SELECT 
        region,
        category,
        COUNT(*) as total_sales,
        SUM(quantity) as total_quantity,
        SUM(quantity * unit_price) as total_revenue,
        AVG(quantity * unit_price) as avg_transaction_value
    FROM sales
    GROUP BY region, category
""")

print("✅ Managed table 'sales_summary' created!")

# Query the table
result12 = spark.sql("SELECT * FROM sales_summary ORDER BY total_revenue DESC")
result12.show(truncate=False)

print("\n2️⃣ Create a table with explicit schema:")
spark.sql("""
    CREATE OR REPLACE TABLE customer_metrics (
        customer_name STRING,
        city STRING,
        membership_level STRING,
        total_orders BIGINT,
        total_spent DOUBLE,
        avg_order_value DOUBLE
    ) USING DELTA
    AS
    SELECT 
        c.customer_name,
        c.city,
        c.membership_level,
        COUNT(s.sale_id) as total_orders,
        COALESCE(SUM(s.quantity * s.unit_price), 0) as total_spent,
        COALESCE(AVG(s.quantity * s.unit_price), 0) as avg_order_value
    FROM customers c
    LEFT JOIN sales s
        ON c.customer_name = s.customer_name
    GROUP BY c.customer_name, c.city, c.membership_level
""")

print("✅ Table 'customer_metrics' created with explicit schema!")

# Query the table
result13 = spark.sql("SELECT * FROM customer_metrics ORDER BY total_spent DESC")
result13.show(truncate=False)

# List all tables
print("\n📋 All available tables:")
spark.sql("SHOW TABLES").show()


## Exercise 6: User Defined Functions (UDF)

### 🎯 **Learning Objectives:**
- Create User Defined Functions (UDFs)
- Register UDFs for use in SQL
- Understand UDF performance considerations


In [ ]:
# User Defined Functions (UDF)
print("⚙️ Exercise 6: User Defined Functions (UDF)")
print("=" * 60)

print("\n1️⃣ Create a simple UDF:")
def calculate_discount(total_amount):
    """Calculate discount based on total amount"""
    if total_amount > 1000:
        return total_amount * 0.1
    elif total_amount > 500:
        return total_amount * 0.05
    else:
        return 0.0

# Register UDF
from pyspark.sql.types import DoubleType
calculate_discount_udf = spark.udf.register("calculate_discount", calculate_discount, DoubleType())

print("✅ UDF 'calculate_discount' registered!")

# Use UDF in SQL
result14 = spark.sql("""
    SELECT 
        sale_id,
        customer_name,
        quantity * unit_price as total_amount,
        calculate_discount(quantity * unit_price) as discount,
        quantity * unit_price - calculate_discount(quantity * unit_price) as final_amount
    FROM sales
    WHERE quantity * unit_price > 500
    ORDER BY total_amount DESC
    LIMIT 10
""")
result14.show(truncate=False)

print("\n2️⃣ Create a UDF for string manipulation:")
def format_customer_name(name, membership):
    """Format customer name with membership level"""
    return f"{name} ({membership})"

from pyspark.sql.types import StringType
format_customer_udf = spark.udf.register("format_customer", format_customer_name, StringType())

print("✅ UDF 'format_customer' registered!")

# Use UDF in SQL
result15 = spark.sql("""
    SELECT 
        format_customer(c.customer_name, c.membership_level) as formatted_name,
        COUNT(s.sale_id) as total_orders,
        SUM(s.quantity * s.unit_price) as total_spent
    FROM customers c
    LEFT JOIN sales s
        ON c.customer_name = s.customer_name
    GROUP BY c.customer_name, c.membership_level
    ORDER BY total_spent DESC
""")
result15.show(truncate=False)


## Exercise 7: Catalog Management

### 🎯 **Learning Objectives:**
- Understand Spark catalog system
- List databases, tables, and functions
- Manage database and table metadata


In [ ]:
# Catalog Management
print("🗂️ Exercise 7: Catalog Management")
print("=" * 60)

print("\n1️⃣ List all databases:")
spark.sql("SHOW DATABASES").show()

print("\n2️⃣ Show current database:")
print(f"Current database: {spark.catalog.currentDatabase()}")

print("\n3️⃣ List all tables in current database:")
spark.sql("SHOW TABLES").show()

print("\n4️⃣ Describe a table:")
spark.sql("DESCRIBE EXTENDED sales_summary").show(truncate=False)

print("\n5️⃣ List all functions:")
print("Available functions (first 20):")
spark.sql("SHOW FUNCTIONS").show(20, truncate=False)

print("\n6️⃣ List user-defined functions:")
print("User-defined functions:")
spark.sql("SHOW USER FUNCTIONS").show()

print("\n7️⃣ Get table details using catalog API:")
tables = spark.catalog.listTables()
print(f"\nTotal tables: {len(tables)}")
for table in tables:
    print(f"  - {table.name} (type: {table.tableType})")


## Summary

### ✅ What we learned:
1. **Spark SQL Basics**: Query DataFrames using SQL syntax
2. **JOIN Operations**: INNER JOIN, LEFT JOIN with SQL
3. **Window Functions**: ROW_NUMBER, RANK, running totals, moving averages
4. **CREATE VIEW**: Create reusable temporary views
5. **CREATE TABLE**: Create managed tables from queries
6. **User Defined Functions**: Create and register UDFs for SQL
7. **Catalog Management**: Manage databases, tables, and functions

### 🎯 Key Takeaways:
- Spark SQL provides a familiar SQL interface for Spark
- Views simplify complex queries and improve reusability
- UDFs extend SQL functionality with custom logic
- Catalog API helps manage metadata and discover resources

### 🚀 Next Steps:
- Practice with more complex SQL queries
- Explore advanced SQL features (CTEs, subqueries)
- Learn about Spark SQL performance optimization
- Integrate Spark SQL with external data sources
